In [3]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('lending-club-data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [6]:
data.shape

(122607, 68)

In [7]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

## create target column

In [8]:
data['safe_loans'] = data['bad_loans'].apply( lambda x: +1 if x==0 else -1 )
data.drop('bad_loans',axis=1,inplace=True)

In [9]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans',
       'emp_length_num', 'grade_num', 'sub_grade_num', 'del

In [10]:
ctr=0
for i in range(len(data)):
    if(data['safe_loans'][i]==1):
        ctr+=1
ctr

99457

In [11]:
pos = ( ctr/len(data) ) * 100
pos

81.11853319957262

In [12]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
data = data[features + [target]]

In [13]:
with open('module-5-assignment-1-train-idx.json','r') as f:
    train_idx = json.load(f)
with open('module-5-assignment-1-validation-idx.json','r') as f:
    test_idx = json.load(f)

In [14]:
train = data.iloc[train_idx]
test = data.iloc[test_idx]
print(train.shape)
print(test.shape)

(37224, 13)
(9284, 13)


In [15]:
safe = train[train['safe_loans']==1]
risky = train[train['safe_loans']==-1]
print(len(safe))
print(len(risky))

18748
18476


In [16]:
# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky)/float(len(safe))

safe = safe.sample(frac=percentage, random_state=0)

print(len(safe))
print(len(risky))

# Append the risky_loans with the downsampled version of safe_loans
data = risky.append(safe)


18476
18476


## One hot encoding

In [17]:
df = pd.get_dummies(data)

In [18]:
df.head()

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
1,1,1,1.00,1,1,9.4,0.0,-1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,5,5.55,1,1,32.6,0.0,-1,0,0,...,0,0,0,0,0,1,0,0,0,1
7,1,1,18.08,1,1,36.5,0.0,-1,0,1,...,0,0,0,0,1,0,0,0,0,1
10,1,1,10.08,1,1,91.7,0.0,-1,0,0,...,0,0,0,0,0,0,0,0,1,0
12,0,4,7.06,1,1,55.5,0.0,-1,0,1,...,0,0,0,0,1,0,0,0,1,0


## Decision tree classifier

In [106]:
from sklearn.tree import DecisionTreeClassifier

In [107]:
model1 = DecisionTreeClassifier(max_depth=6)
model2 = DecisionTreeClassifier(max_depth=2)

In [120]:
train_ = pd.get_dummies(train)
test_ = pd.get_dummies(test)

In [121]:
cols = train_.columns
cols=['short_emp', 'emp_length_num', 'dti', 'last_delinq_none',
       'last_major_derog_none', 'revol_util', 'total_rec_late_fee', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E',
       'grade_F', 'grade_G', 'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3',
       'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2',
       'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1',
       'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5',
       'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4',
       'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3',
       'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2',
       'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1',
       'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'purpose_car', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_home_improvement',
       'purpose_house', 'purpose_major_purchase', 'purpose_medical',
       'purpose_moving', 'purpose_other', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months']
tar = 'safe_loans'

In [122]:
x_train = train_.loc[:,cols]
y_train = train_.loc[:,tar]
x_test = test_.loc[:,cols]
y_test = test_.loc[:,tar]

In [123]:
print(train_.shape)
print(x_train.shape)
print(y_train.shape)

(37224, 68)
(37224, 67)
(37224,)


In [124]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Visualize small tree

In [125]:
import graphviz
from sklearn import tree
from IPython.display import display

In [126]:
tree.export_graphviz(model2, 
                  #feature_names=x_train.columns,  
                  #class_names=['1','-1'],  
                  filled=True, rounded=True,  
                  special_characters=True,
                   out_file='tree.dot',
                           )


In [129]:
with open("tree.dot") as f:
    dot_graph = f.read()
#graphviz.Source(dot_graph)

### Make predictions

In [130]:
validation_safe_loans = test_[test_[target] == 1]
validation_risky_loans = test_[test_[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
19,0,11,11.18,1,1,82.4,0.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
79,0,10,16.85,1,1,96.4,0.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
24,0,3,13.97,0,1,59.5,0.0,-1,0,0,...,0,0,0,0,1,0,0,0,0,1
41,0,11,16.33,1,1,62.1,0.0,-1,1,0,...,0,0,0,0,0,0,0,0,1,0


In [133]:
x = sample_validation_data.loc[:,cols]
y = sample_validation_data.loc[:,tar]
pred1 = model1.predict(x)
print(pred1)
print(list(y))

[ 1 -1 -1  1]
[1, 1, -1, -1]


### Prob prediction

In [134]:
pred2 = model2.predict(x)
print(pred2)
print(list(y))
model2.predict_proba(x)

[ 1 -1 -1  1]
[1, 1, -1, -1]


array([[0.41896585, 0.58103415],
       [0.59255339, 0.40744661],
       [0.59255339, 0.40744661],
       [0.23120112, 0.76879888]])

### Visualise pred on tree

### Accuracy

In [142]:
score1 = model1.score(x_train,y_train)
score2 = model2.score(x_train,y_train)
print(score1)
print(score2)

0.6405276165914464
0.6135020416935311


In [143]:
score1 = model1.score(x_test,y_test)
score2 = model2.score(x_test,y_test)
print(score1)
print(score2)

0.6363636363636364
0.6193451098664369


### Complex decision tree

In [144]:
model3 = DecisionTreeClassifier(max_depth=10)
model3.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [145]:
score3 = model3.score(x_train,y_train)
score3

0.6637921770900495

In [146]:
score3 = model3.score(x_test,y_test)
score3

0.6265618267987936

### Cost of mistake

In [166]:
def costcalc(pred,label):
    false_pos=0
    false_neg=0
    for i in range(len(pred)):
        if(pred[i]==-1 and label[i]==1):
            false_neg+=1
        elif(pred[i]==1 and label[i]==-1):
            false_pos+=1
    print(false_neg)
    print(false_pos)
    cost = (false_neg*10000) + (false_pos*20000)
    return cost

In [167]:
pred = model1.predict(x_test)
cost = costcalc(pred,list(y_test))
cost

1715
1661


50370000